In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import time

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Input, Embedding, Flatten
from keras import Model
from keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import relu, elu, sigmoid, softmax
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical


import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

In [14]:
# Load Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

#x_train
pickle_in = open(os.path.join(pickle_path,'x_train.pickle'),"rb")
x_train = pickle.load(pickle_in)

#y_train
pickle_in = open(os.path.join(pickle_path,'y_train.pickle'),"rb")
y_train = pickle.load(pickle_in)

#x_val
pickle_in = open(os.path.join(pickle_path,'x_dev.pickle'),"rb")
x_val = pickle.load(pickle_in)

#y_val
pickle_in = open(os.path.join(pickle_path,'y_dev.pickle'),"rb")
y_val = pickle.load(pickle_in)

#x_test
pickle_in = open(os.path.join(pickle_path,'x_test.pickle'),"rb")
x_test = pickle.load(pickle_in)

#y_test
pickle_in = open(os.path.join(pickle_path,'y_test.pickle'),"rb")
y_test = pickle.load(pickle_in)

# convert y values to one-hot encoding
y_train = to_categorical(y_train - y_train.min())
y_val = to_categorical(y_val - y_val.min())
y_test = to_categorical(y_test - y_test.min())

In [47]:
def sentiment_model(x_train, y_train, x_val, y_val, params):
    
    model = Sequential()
    model.add(Bidirectional(LSTM(params['first_neuron'], activation=params['activation'], dropout=params['dropout'], return_sequences=True), input_shape=(80,512)))
    model.add(Bidirectional(LSTM(25,activation='relu',dropout=params['dropout'], return_sequences=False)))
    model.add(Dense(params['dense_1_units'], activation = params['activation']))
    model.add(Dense(params['dense_2_units'], activation = params['last_activation']))
    model.compile(optimizer=params['optimizer'],loss=params['losses'], metrics='accuracy')


    batch_num = params['batch_size']
    # model name
#     NAME = "biLSTM_L1={}_L2={}_d1={}_d2={}_drop={}_{}".format(params['lstm_1_units'],params['lstm_2_units'],params['dense_1_units'],params['dense_2_units'],params['dropout'],int(time.time()))

#     print("######################################################")
#     print(NAME)

#     # callbacks
#     tensorboard = TensorBoard(log_dir="logs/biLSTM1/{}".format(NAME))


    out =model.fit(x_train, y_train,
                epochs = params['epochs'],
                validation_data=(x_val,y_val),
                shuffle=True,
                batch_size=batch_num,
                verbose = 0,
                callbacks=[live()])

    return out, model

In [48]:
# then we can go ahead and set the parameter space

p = {'first_neuron':[50, 100, 20],
     'dense_1_units':[50,100],
     'dense_2_units':[7],
     'batch_size': [30],
     'epochs': [100],
     'dropout': [0],
     'optimizer': ['adam'],
     'losses': [categorical_crossentropy],
     'activation':[relu],
     'last_activation': [softmax]}

In [49]:
t = ta.Scan(x=x_train,
            y=y_train,
            x_val=x_val,
            y_val=y_val,
            model=sentiment_model,
            dataset_name='breast_cancer',
            experiment_no='1',
            params=p,)












  0%|          | 0/6 [00:00<?, ?it/s]

ERROR MESSAGE : Type of `metrics` argument not understood. Expected a list or dictionary, found: accuracy


TalosReturnError: Make sure that input model returns 'out, model' where out is history object from model.fit()